In [17]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

import pickle
from tqdm import tqdm
from topolearn import *
import warnings
from sklearn.model_selection import train_test_split
import scipy.io
import numpy as np
warnings.filterwarnings("ignore")

from lib import *
import pandas as pd
import scipy.io
from sklearn.preprocessing import normalize 
from collections import defaultdict
import numpy.linalg as la



def compute_error(Y,X):
    num_col = Y.shape[1]
    error = 0
    for col in range(num_col):
        error = error + np.linalg.norm(Y[:,col]-X[:,col])**2/np.linalg.norm(Y[:,col])**2
    return (1/num_col)*error

def nmse(D, X, Y, m):
    return (1 / m) * np.sum(la.norm(Y - (D @ X), axis=0) ** 2 / la.norm(Y, axis=0) ** 2)

#~~~~~~~~~~~~~~~#
# data loading  #
#~~~~~~~~~~~~~~~#


mat = scipy.io.loadmat('C:\\Users\\engri\\Desktop\\tesi\\TSP-DictionaryLearning\\real_data\\data_real.mat')
# edge_signals = np.array(mat["signal_edge"].T, dtype=float)
# valid_signal = np.where(np.sum(edge_signals, axis=1)!=0)
# edge_signals = edge_signals[valid_signal]
flow = mat["signal_edge"][:,~np.all(mat["signal_edge"] == 0, axis = 0)]

# flow = flow/np.max(flow)
# flow = flow[:,10:]

In [16]:
flow.shape

(89, 26)

In [21]:

B1 = mat["B1"]
B2 = mat["B2"]
N0 = B1.shape[0]
N1 = B1.shape[1]
N2 = B2.shape[1]
#B1, B2 = scale_incidence_matrices(B1, B2)
#sgn_change = np.diag(np.sign(np.random.randn(N1)))
#B1 = B1@sgn_change
#B2 = sgn_change@B2
L, L1, L2 = hodge_laplacians(B1, B2)


w = np.linalg.eigvalsh(L)
w1 = np.linalg.eigvalsh(L1)
w2 = np.linalg.eigvalsh(L2)

L_line = L.copy()
L_line = -(np.abs(L_line))
np.fill_diagonal(L_line, 0)
L_line -= np.diag(np.sum(L_line, axis=0))
w_line = np.linalg.eigvalsh(L_line)

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# Wavelet, Fourier and Slepians making #
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

W_fourier = FourierBasis(B1, B2)
W_joint = JointHodgelet(B1, B2,
                        *log_wavelet_kernels_gen(3, 4, np.log(np.max(w))))
W_sep = SeparateHodgelet(B1, B2,
                         *log_wavelet_kernels_gen(3, 4, np.log(np.max(w1))),
                         *log_wavelet_kernels_gen(3, 4, np.log(np.max(w2))))
W_lift = LiftedHodgelet(B1, B2,
                        *log_wavelet_kernels_gen(3, 4, np.log(np.max(w1))),
                        *log_wavelet_kernels_gen(3, 4, np.log(np.max(w2))))
W_lift_mixed = MixedLiftedHodgelet(B1, B2,
                                   *log_wavelet_kernels_gen(3, 4, np.log(np.max(w1))),
                                   *log_wavelet_kernels_gen(3, 4, np.log(np.max(w2))))

W_fourier_line = LaplaceFourierBasis(L_line)
W_line = JointLaplacelet(L_line,
                         *log_wavelet_kernels_gen(3, 4, np.log(np.max(w_line))))


option = "One-shot-diffusion"#"One-shot-diffusion"
F_sol,F_irr = get_frequency_mask(B1,B2) # Get frequency bands
diff_order_sol= 1
diff_order_irr = 1
step_prog = 1
source_sol = np.ones((N1,))
source_irr = np.ones((N1,))
S_neigh, complete_coverage = cluster_on_neigh(B1,B2,diff_order_sol,diff_order_irr,source_sol,source_irr,option,step_prog)
R = [F_sol, F_irr]
S = S_neigh
top_K_coll = [2,4, None]
spars_level = np.arange(5, 26, 4)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# orthogonal matching pursuit #
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
error_data = defaultdict(list)    
for top_K_slep in top_K_coll:
    print("Complete Coverage: "+str(complete_coverage))
    W_slepians = SimplicianSlepians(B1, B2, S, R, top_K = top_K_slep, save_dir = res_dir, load_dir = res_dir)
    print("Slepians Rank: "+str(W_slepians.full_rank))
    print("Dictionary Dimension: "+str(W_slepians.atoms_flat.shape))
    slepians_noisy = [W_slepians.omp(flow, k = spars)
                            for spars in spars_level]
    slepians_error = [compute_error(flow, W_slepians.atoms_flat@spars.coef_.T)
                            for spars in slepians_noisy]
    error_data["slepians "+str(top_K_slep)]= slepians_error

fourier_noisy = [W_fourier.omp(flow, k = spars)
                        for spars in spars_level]
fourier_error = [compute_error(flow, W_fourier.atoms_flat@spars.coef_.T)
                                for spars in fourier_noisy]

sep_noisy = [W_sep.omp(flow, k = spars)
                        for spars in spars_level]
sep_error = [compute_error(flow, W_sep.atoms_flat@spars.coef_.T)
                                for spars in sep_noisy]

error_data['fourier']=fourier_error
error_data['sep']=sep_error
"""   
error_data = {**error_data,**complete_coll_error}
for key in error_data.keys():
    error_data[key] = np.mean(np.array(error_data[key]),axis = 0)
"""
print(error_data)
error_df = pd.DataFrame(error_data,
                            columns=list(error_data.keys()),
                            index=spars_level)
print(error_df)
#error_df.to_csv(f'{res_dir}/error_snr_'+str(snr)+'.csv', float_format='%0.4f', index_label='err', sep = ";")



Complete Coverage: 1
Harmonic Component is Present!
Saving Directory not Valid!
Slepians Rank: 1
Dictionary Dimension: (89, 337)
Complete Coverage: 1
Harmonic Component is Present!
Saving Directory not Valid!
Slepians Rank: 1
Dictionary Dimension: (89, 657)
Complete Coverage: 1
Harmonic Component is Present!
Saving Directory not Valid!
Slepians Rank: 1
Dictionary Dimension: (89, 1012)
defaultdict(<class 'list'>, {'slepians 2': [0.19231625071006472, 0.1166996740022137, 0.0777999663220373, 0.05347708775561865, 0.03712373968055475, 0.026004664369903948], 'slepians 4': [0.1918585488845948, 0.11147942380765813, 0.06246827266434132, 0.03604342086818035, 0.019991805606688926, 0.01141867415204933], 'slepians None': [0.18329278240044805, 0.08427877006293673, 0.040088485789576044, 0.02174060920787337, 0.012342665069467799, 0.00663445313254727], 'fourier': [0.5951997110399401, 0.45669814073853754, 0.342856761093417, 0.2594853541169462, 0.20550531947575137, 0.1635147997154655], 'sep': [0.236961283

In [19]:
spars_level

[10, 20, 30, 40, 50, 60, 70]

In [4]:
from lib import *
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import sys
import time

res_dir = "/Users/Claudio/Desktop/Topological_Slepians/Results"

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#
# setup - data creation function #
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~#

def hexgrid(n=15):

    xs = np.linspace(-2,2,n)
    ys = np.sqrt(3)*np.linspace(-2,2,n)
    
    xint = (xs[1]-xs[0])/2
    
    x1, y1 = np.meshgrid(xs,ys)
    x1[::2] += xint
    x1 = x1.flatten()
    y1 = y1.flatten()
    
    coords = np.vstack([x1,y1]).T

    # sort coords
    diagonal_coordinates = np.sum(coords, axis=1)  # y = -x + c, compute c
    diagonal_idxs = np.argsort(diagonal_coordinates)  # sort by c: origin comes first, upper-right comes last
    coords = coords[diagonal_idxs]  # apply sort to original coordinates
    
    tri = Delaunay(coords)
    simplices = [sorted(f) for f in tri.simplices]
    
    edges = []
    for f in simplices:
        [a,b,c] = sorted(f)
        edges.append((a,b))
        edges.append((b,c))
        edges.append((a,c))
    edges = sorted(set(edges))
    edges_tup = [tuple(e) for e in edges]

    edge_idx_dict = {edges_tup[i]: i for i in range(len(edges_tup))}

    nodes = list(range(len(coords)))

    graph = nx.DiGraph()
    graph.add_nodes_from(nodes)
    graph.add_edges_from(edges)

    return graph, nodes, edges_tup, simplices, coords, edge_idx_dict

def incidence_matrices(G, V, E, faces, edge_to_idx):
    # redefined from nblib.py
    B1 = np.array(nx.incidence_matrix(G, nodelist=V, edgelist=E, oriented=True).todense())
    B2 = np.zeros([len(E),len(faces)])

    for f_idx, face in enumerate(faces): # face is sorted
        edges = [face[:-1], face[1:], [face[0], face[2]]]
        e_idxs = [edge_to_idx[tuple(e)] for e in edges]

        B2[e_idxs[:-1], f_idx] = 1
        B2[e_idxs[-1], f_idx] = -1
    return B1, B2

def discrete_flow(E, coords):

    flow_dict = {}

    A = np.array([[0.5, -1/(4*np.sqrt(3))],[1/(4*np.sqrt(3)), 0.5]])

    for e in E:
        j0,j1 = e
        p0 = coords[j0]
        p1 = coords[j1]

        clipradius = 0.7
        if (np.linalg.norm(0.5*(p0+p1)-np.array([np.pi/4,np.pi/4]))>clipradius) and \
           (np.linalg.norm(0.5*(p0+p1)-np.array([-np.pi/4,-np.pi/4]))>clipradius):
            flow_dict[e] = 0
            continue

        d = p1 - p0

        q0 = p0 + A @ d
        q1 = p0 + A.T @ d

        # black magic: F = [cos(x+y), sin(x-y)]
        # I integrate perpendicular to the edges between hexagons
        f = ((q0[1]-q1[1])/((q1[0]+q1[1])-(q0[0]+q0[1])) * (np.sin(q1[0]+q1[1])-np.sin(q0[0]+q0[1])) -
             (q0[0]-q1[0])/((q1[0]-q1[1])-(q0[0]-q0[1])) * (np.cos(q1[0]-q1[1])-np.cos(q0[0]-q0[1])))

        flow_dict[e] = f

    return flow_dict

#~~~~~~~~~~~~~~~#
# data building #
#~~~~~~~~~~~~~~~#
G, V, E, F, coords, edge_idx_dict = hexgrid()
N0 = len(V)
N1 = len(E)
N2 = len(F)
B1, B2 = incidence_matrices(G, V, E, F, edge_idx_dict)
B1, B2 = scale_incidence_matrices(B1, B2)
sgn_change = np.diag(np.sign(np.random.randn(N1)))
B1 = B1 @ sgn_change
B2 = sgn_change @ B2
L, L1, L2 = hodge_laplacians(B1, B2)

# Generate Signal
# compute a mapping from edges to the reals
flow_dict = discrete_flow(E, coords)
# then use the indexing to write as a vector in R^E
flow2 = np.zeros(N1)
for e in E:
    flow2[edge_idx_dict[e]] = flow_dict[e]
flow2 /= np.linalg.norm(flow2)

w = np.linalg.eigvalsh(L)
w1 = np.linalg.eigvalsh(L1)
w2 = np.linalg.eigvalsh(L2)

L_line = L.copy()
L_line = -(np.abs(L_line))
np.fill_diagonal(L_line, 0)
L_line -= np.diag(np.sum(L_line, axis=0))
w_line = np.linalg.eigvalsh(L_line)

In [7]:
flow[0].shape

()

# My part


In [24]:
from tsplearn import *
import numpy as np 
import pandas as pd

prob_T=0.8

# Load the graph
G = EnhancedGraph(n=40, p_edges=0.162, p_triangles=prob_T, seed=0)
B1 = G.get_b1()
B2 = G.get_b2()

# Sub-sampling if needed to decrease complexity
sub_size = 100
B1 = B1[:, :sub_size]
B2 = B2[:sub_size, :]
B2 = B2[:,np.sum(np.abs(B2), 0) == 3]
nu = B2.shape[1]
nd = B1.shape[1]
T = int(np.ceil(nu*(1-prob_T)))

# Laplacians
Lu, Ld, L = G.get_laplacians(sub_size=100)
Lu_full = G.get_laplacians(sub_size=100, full=True)
B2_true = B2@G.mask
M =  L.shape[0]


# Problem and Dictionary Dimensionalities
dictionary_type="separated"
m_train = 150 # Number of Train Signals
m_test = 80 # Number of Test Signal
P = 3 # Number of Kernels (Sub-dictionaries)
J = 2 # Polynomial order
sparsity = .1 # Sparsity percentage
K0_max = 20 #floor(M*sparsity) # Sparsity
sparsity_mode = "max"
n_search = 3000
n_sim = 10

# Data-Independent Problem Hyperparameters
K0_coll = np.arange(5, 26, 4) 
max_iter = 100 
patience = 5 
tol = 1e-7 # tolerance for Patience
lambda_ = 1e-7 # l2 multiplier
verbose = True

gen_params={'dictionary_type':dictionary_type,
        'm_train':m_train,
        'm_test':m_test,
        'P':P,
        'M':M,
        'J':J,
        'sparsity':sparsity,
        'K0_max':K0_max,
        'sparsity_mode':sparsity_mode,
        'n_search':n_search,
        'n_sim':n_sim,
        'prob_T':prob_T}

load_data = generate_data(Lu, Ld, **gen_params)

D_true = load_data['D_true']
Y_train = load_data['Y_train']
Y_test = load_data['Y_test']
X_train = load_data['X_train']
X_test = load_data['X_test']
epsilon_true =  load_data['epsilon_true']
c_true = load_data['c_true']

s = 8
c = c_true[s]  
epsilon = epsilon_true[s] 
k0 = K0_coll[4]

topo_params = {"K0":K0_coll[4],
               "J":J,
               "P":P,
               "true_prob_T":prob_T,
               "sub_size":100,
               "dictionary_type":dictionary_type,
               "c":c_true[s],
               "epsilon":epsilon_true[s],
               "seed":0,
               "n":40,
               "p_edges":0.162
               }

ImportError: attempted relative import with no known parent package

In [ ]:
import scipy.linalg as sla
import numpy as np
import numpy.linalg as la
import cvxpy as cp
from typing import Tuple, List, Union, Dict
import pickle
from functools import wraps


import scipy.linalg as sla
import numpy as np
import numpy.linalg as la
import cvxpy as cp
from tsplearn.tsp_generation import *
from typing import Tuple, List, Union, Dict
import pickle
from functools import wraps
from einops import rearrange


def _indicator_matrix(row):
    tmp = row.sigma.copy()
    tmp[row.idx] = 0
    return np.diag(tmp)

def _indicator_matrix_rev(row):
    tmp = row.sigma.copy()
    tmp[row.idx] = 1
    return np.diag(tmp)

def _compute_Luj(row, b2, J):
    Lu = b2 @ row.sigma @ b2.T
    Luj = np.array([la.matrix_power(Lu, i) for i in range(1, J + 1)])
    return Luj

def _split_coeffs(h ,s ,k, sep=False):
    h_tmp = h.value.flatten()
    # hH = h_tmp[:s,].reshape((s,1))
    # hS = h_tmp[s:s*(k+1),].reshape((s,k))
    # hI = h_tmp[s*(k+1):,].reshape((s,k))
    if sep:
        hH = h_tmp[np.arange(0, (s*(2*k+1)), (2*k+1))].reshape((s,1))
        hS = h_tmp[np.hstack([[i,i+1] for i in range(1, (s*(2*k+1)), (2*k+1))])].reshape((s,k))
        hI = h_tmp[np.hstack([[i,i+1] for i in range((k+1), (s*(2*k+1)), (2*k+1))])].reshape((s,k))
        return [hH, hS, hI]
    h = h_tmp[:s*k]
    hi = h_tmp[s*k:]
    return [h, hi]
    
def sparse_transform(D, K0, Y_te, Y_tr=None):

    dd = la.norm(D, axis=0)
    W = np.diag(1. / dd)
    Domp = D @ W
    X_te = np.apply_along_axis(lambda x: get_omp_coeff(K0, Domp=Domp, col=x), axis=0, arr=Y_te)
    # Normalization
    X_te = W @ X_te

    if np.all(Y_tr == None):

        return X_te
    
    # Same for the training set
    X_tr = np.apply_along_axis(lambda x: get_omp_coeff(K0, Domp=Domp, col=x), axis=0, arr=Y_tr)
    X_tr = W @ X_tr
    
    return X_te, X_tr


def compute_vandermonde(L, k):
    
    def polynomial_exp(x, k):
        x = x** np.arange(0, k + 1)
        return x

    eigenvalues, _ = sla.eig(L)
    idx = eigenvalues.argsort()
    tmp_df = pd.DataFrame({'Eigs': eigenvalues[idx]})
    tmp_df['Poly'] = tmp_df['Eigs'].apply(lambda x:  polynomial_exp(x,k))
    B = np.vstack(tmp_df['Poly'].to_numpy())

    return B


def nmse(D, X, Y, m):
    return (1/m)* np.sum(la.norm(Y - (D @ X), axis=0)**2 /la.norm(Y, axis=0)**2)


class TspSolver:

    def __init__(self, X_train, X_test, Y_train, Y_test, *args, **kwargs):

        params = {
                'P': None,      # Number of Kernels (Sub-dictionaries)
                'J': None,      # Polynomial order
                'K0': None,     # Sparsity level
                'dictionary_type': None,
                'c': None,      # spectral control parameter 
                'epsilon': None,# spectral control parameter
                'n': 10,        # number of nodes
                'sub_size': None,   # Number of sub-sampled nodes
                'prob_T': 1.,   # Ratio of colored triangles
                'true_prob_T': 1.,   # True ratio of colored triangles
                'p_edges': 1.,  # Probability of edge existence
                'seed': None
                }
        
        if args:
            if len(args) != 1 or not isinstance(args[0], dict):
                raise ValueError("When using positional arguments, must provide a single dictionary")
            params.update(args[0])

        params.update(kwargs)

        # Data
        self.X_train: np.ndarray = X_train
        self.X_test: np.ndarray = X_test
        self.Y_train: np.ndarray = Y_train
        self.Y_test: np.ndarray = Y_test
        self.m_train: int = Y_train.shape[1]
        self.m_test: int = Y_test.shape[1]

        # Topology and geometry behind data
        self.G = EnhancedGraph(n=params['n'],
                               p_edges=params['p_edges'], 
                               p_triangles=params['prob_T'], 
                               seed=params['seed'])
        self.B1: np.ndarray = self.G.get_b1()
        self.B2: np.ndarray = self.G.get_b2()

        # Sub-sampling if needed to decrease complexity
        if params['sub_size'] != None:
            self.B1 = self.B1[:, :params['sub_size']]
            self.B2 = self.B2[:params['sub_size'], :]
            self.B2 = self.B2[:,np.sum(np.abs(self.B2), 0) == 3]
        self.nu: int = self.B2.shape[1]
        self.nd: int = self.B1.shape[1]
        self.true_prob_T = params['true_prob_T']
        self.T: int = int(np.ceil(self.nu*(1-params['prob_T'])))

        # Laplacians
        Lu, Ld, L = self.G.get_laplacians(sub_size=params['sub_size'])
        self.Lu: np.ndarray = Lu
        self.Ld: np.ndarray = Ld
        self.L: np.ndarray = L
        self.Lu_full: np.ndarray = G.get_laplacians(sub_size=params['sub_size'], 
                                                    full=True)
        self.M =  L.shape[0]
        self.history: List[np.ndarray] = []

        # Dictionary, parameters and hyperparameters for compression
        self.P = params['P']
        self.J = params['J']
        self.c = params['c']
        self.epsilon = params['epsilon']
        self.K0 = params['K0']
        self.dictionary_type = params['dictionary_type']
        self.D_opt: np.ndarray = np.zeros((self.M, self.M*self.P))

        if self.dictionary_type=="separated":
            hs = np.zeros((self.P,self.J))
            hi = np.zeros((self.P,self.J))
            hh = np.zeros((self.P,1))
            self.h_opt: List[np.ndarray] = [hh,hs,hi]
        else:
            h = np.zeros((self.P, self.J))
            hI = np.zeros((self.P, 1))
            self.h_opt: np.ndarray = [h, hI]
            
        self.X_opt_train: np.ndarray = np.zeros(self.X_train.shape)
        self.X_opt_test: np.ndarray = np.zeros(self.X_test.shape)

        if self.dictionary_type == "joint":
            self.Lj, self.lambda_max_j, self.lambda_min_j = compute_Lj_and_lambdaj(self.L, self.J)
            self.B = compute_vandermonde(self.L, self.J).real
        elif self.dictionary_type == "edge_laplacian":
            self.Lj, self.lambda_max_j, self.lambda_min_j = compute_Lj_and_lambdaj(self.Ld, self.J)
            self.B = compute_vandermonde(self.Ld, self.J).real
        elif  self.dictionary_type == 'separated':
            self.Luj, self.lambda_max_u_j, self.lambda_min_u_j = compute_Lj_and_lambdaj(self.Lu, self.J, separated=True)
            self.Ldj, self.lambda_max_d_j, self.lambda_min_d_j = compute_Lj_and_lambdaj(self.Ld, self.J, separated=True)
            self.Bu = compute_vandermonde(self.Lu, self.J).real
            self.Bd = compute_vandermonde(self.Ld, self.J)[:, 1:].real
            self.B = np.hstack([self.Bu, self.Bd])

        self.P_aux: np.ndarray = None

        # Init the learning errors
        self.min_error_train = 1e20
        self.min_error_test = 1e20

    # def fit(self) -> Tuple[float, List[np.ndarray], np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
    #     min_error_test, _, _, h_opt, X_opt_test, D_opt = self.learn_upper_laplacian()
    #     return min_error_test, self.history, params['Lu'], h_opt, X_opt_test, D_opt

    def update_Lu(self, Lu_new):
        self.Lu = Lu_new
        self.Luj, self.lambda_max_u_j, self.lambda_min_u_j = compute_Lj_and_lambdaj(self.Lu, 
                                                                                    self.J, 
                                                                                    separated=True)
        self.Bu = compute_vandermonde(self.Lu, self.J).real
        self.B = np.hstack([self.Bu, self.Bd])

    @staticmethod
    def _multiplier_search(*arrays, P, c, epsilon):
        is_okay = 0
        mult = 100
        tries = 0
        while is_okay==0:
            is_okay = 1
            h, c_try, _, tmp_sum_min, tmp_sum_max = generate_coeffs(arrays, P=P, mult=mult)
            if c_try <= c:
                is_okay *= 1
            if tmp_sum_min > c-epsilon:
                is_okay *= 1
                incr_mult = 0
            else:
                is_okay = is_okay*0
                incr_mult = 1
            if tmp_sum_max < c+epsilon:
                is_okay *= 1
                decr_mult = 0
            else:
                is_okay *= 0
                decr_mult = 1
            if is_okay == 0:
                tries += 1
            if tries >3:
                discard = 1
                break
            if incr_mult == 1:
                mult *= 2
            if decr_mult == 1:
                mult /= 2
        return h, discard

    def init_dict(self,
                  h_prior: np.ndarray = None, 
                  mode: str = "only_X") -> Tuple[np.ndarray, np.ndarray]:
        """
        Initialize the dictionary and the signal sparse representation for the alternating
        optimization algorithm.

        Args:
            Lu (np.ndarray): Upper Laplacian matrix
            Ld (np.ndarray): Lower Laplacian matrix
            P (int): Number of kernels (sub-dictionaries).
            J (int): Max order of the polynomial for the single sub-dictionary.
            Y_train (np.ndarray): Training data.
            K0 (int): Sparsity of the signal representation.
            dictionary_type (str): Type of dictionary.
            c (float): Boundary constant from the synthetic data generation process.
            epsilon (float): Boundary constant from the synthetic data generation process.
            only (str): Type of initialization. Can be one of: "only_X", "all", "only_D".

        Returns:
            Tuple[np.ndarray, np.ndarray, bool]: Initialized dictionary, initialized sparse representation, and discard flag value.
        """
        
        # If no prior info on the dictionary
        if np.all(h_prior == None):

            if (mode in ["all","only_D"]):

                discard = 1
                while discard==1:

                    if dictionary_type != "separated":
                        h_prior, discard = self._multiplier_search(self.lambda_max_j, 
                                                              self.lambda_min_j, 
                                                              P=self.P, 
                                                              c=self.c, 
                                                              epsilon=self.epsilon)
                        self.D_opt = generate_dictionary(h_prior, 
                                                         self.P, 
                                                         self.Lj)

                    else:
                        h_prior, discard = self._multiplier_search(self.lambda_max_d_j, 
                                                              self.lambda_min_d_j, 
                                                              self.lambda_max_u_j, 
                                                              self.lambda_min_u_j,
                                                              P=self.P, 
                                                              c=self.c, 
                                                              epsilon=self.epsilon)
                        self.D_opt = generate_dictionary(h_prior, 
                                                         self.P, 
                                                         self.Luj, 
                                                         self.Ldj)
        
            if (mode in ["all","only_X"]):
                
                L = self.Ld if self.dictionary_type == "edge_laplacian" else self.L
                _, Dx = sla.eig(L)
                dd = la.norm(Dx, axis=0)
                W = np.diag(1./dd)
                Dx = Dx / la.norm(Dx)  
                Domp = Dx@W
                X = np.apply_along_axis(lambda x: get_omp_coeff(self.K0, Domp.real, x), axis=0, arr=self.Y_train)
                X = np.tile(X, (self.P,1))
                self.X_opt_train = X

        # Otherwise use prior info about the dictionary to initialize the sparse representation
        else:
            
            self.h_opt = h_prior

            if dictionary_type == "separated":
                self.D_opt = generate_dictionary(h_prior, 
                                                 self.P, 
                                                 self.Luj, 
                                                 self.Ldj)
                self.X_opt_train = sparse_transform(self.D_opt, 
                                                    self.K0, 
                                                    self.Y_train)
            else: 
                self.D_opt = generate_dictionary(h_prior, 
                                                 self.P, 
                                                 self.Lj)
                self.X_opt_train = sparse_transform(self.D_opt, 
                                                    self.K0, 
                                                    self.Y_train)             

   
    def save_results(func):
        @wraps(func)
        def wrapper(self, *args, **kwargs):

            outputs = func(self, *args, **kwargs)
            func_name = func.__name__

            if func_name == "topological_dictionary_learn":

                path = os.getcwd()
                dir_path = os.path.join(path, 
                                        'results', 
                                        'dictionary_learning',
                                        f'{self.dictionary_type}')
                name = f'learn_D_{self.dictionary_type}'
                filename = os.path.join(dir_path, f'{name}.pkl')
                save_var = {"min_error_test": self.min_error_test,
                            "min_error_train": self.min_error_train,
                            "history": outputs[2],
                            "h_opt": self.h_opt,
                            "X_opt_test": self.X_opt_test,
                            "X_opt_train": self.X_opt_train,
                            "D_opt": self.D_opt}
                
            elif func_name == "learn_upper_laplacian":

                path = os.getcwd()
                dir_path = os.path.join(path, 'results', 'topology_learning')
                name = f'learn_T{int(self.true_prob_T*100)}'
                filename = os.path.join(dir_path, f'{name}.pkl')
                save_var = {"min_error_test": self.min_error_test,
                            "min_error_train": self.min_error_train,
                            "history": self.history,
                            "Lu_opt": self.Lu,
                            "h_opt": self.h_opt,
                            "X_opt_test": self.X_opt_test,
                            "X_opt_train": self.X_opt_train,
                            "D_opt": self.D_opt}

            if not os.path.exists(dir_path):
                os.makedirs(dir_path)

            try:
                with open(filename, 'wb') as file:
                    pickle.dump(save_var, file)
            except IOError as e:
                print(f"An error occurred while writing the file: {e}")
            except Exception as e:
                print(f"An unexpected error occurred: {e}")

            return outputs  
        return wrapper

    def topological_dictionary_learn(self,
                                     lambda_: float = 1e-3, 
                                     max_iter: int = 10, 
                                     patience: int = 10,
                                     tol: float = 1e-7,
                                     step_h: float = 1.,
                                     step_x: float = 1.,
                                     solver: str ="MOSEK", 
                                     verbose: bool = False) -> Tuple[np.ndarray, np.ndarray, List[float]]:
        """
        Dictionary learning algorithm implementation for sparse representations of a signal on complex regular cellular.
        The algorithm consists of an iterative alternating optimization procedure defined in two steps: the positive semi-definite programming step
        for obtaining the coefficients and dictionary based on Hodge theory, and the Orthogonal Matching Pursuit step for constructing 
        the K0-sparse solution from the dictionary found in the previous step, which best approximates the original signal.
        Args:
            Y_train (np.ndarray): Training data.
            Y_test (np.ndarray): Testing data.
            J (int): Max order of the polynomial for the single sub-dictionary.
            M (int): Number of data points (number of nodes in the data graph).
            P (int): Number of kernels (sub-dictionaries).
            D0 (np.ndarray): Initial dictionary.
            X0 (np.ndarray): Initial sparse representation.
            Lu (np.ndarray): Upper Laplacian matrix
            Ld (np.ndarray): Lower Laplacian matrix
            dictionary_type (str): Type of dictionary.
            c (float): Boundary constant from the synthetic data generation process.
            epsilon (float): Boundary constant from the synthetic data generation process.
            K0 (int): Sparsity of the signal representation.
            lambda_ (float, optional): Regularization parameter. Defaults to 1e-3.
            max_iter (int, optional): Maximum number of iterations. Defaults to 10.
            patience (int, optional): Patience for early stopping. Defaults to 10.
            tol (float, optional): Tolerance value. Defaults to 1e-s.
            verbose (int, optional): Verbosity level. Defaults to 0.

        Returns:
            Tuple[np.ndarray, np.ndarray, np.ndarray, np.ndarray, np.ndarray]:
            minimum training error, minimum testing error, optimal coefficients, optimal testing sparse representation, and optimal training sparse representation.
        """

        # Define hyperparameters
        iter_, pat_iter = 1, 0
        hist = []

        if self.dictionary_type != "fourier":

            # Init the dictionary and the sparse representation 
            D_coll = [cp.Constant(self.D_opt[:,(self.M*i):(self.M*(i+1))]) for i in range(self.P)]
            Dsum = cp.Constant(np.zeros((self.M, self.M)))
            h_opt = self.h_opt
            Y = cp.Constant(self.Y_train)
            X_tr = self.X_opt_train
            X_te = self.X_opt_test
            I = cp.Constant(np.eye(self.M))
            
            while pat_iter < patience and iter_ <= max_iter:
                
                # SDP Step
                X = cp.Constant(X_tr)
                if iter_ != 1:
                    D_coll = [cp.Constant(D[:,(self.M*i):(self.M*(i+1))]) for i in range(self.P)]
                    Dsum = cp.Constant(np.zeros((self.M, self.M)))
                
                # Define the objective function
                if self.dictionary_type in ["joint", "edge_laplacian"]:
                    # Init the variables
                    h = cp.Variable((self.P, self.J))
                    hI = cp.Variable((self.P, 1))
                    h.value, hI.value = h_opt
                    for i in range(0,self.P):
                        tmp =  cp.Constant(np.zeros((self.M, self.M)))
                        for j in range(0,self.J):
                            tmp += (cp.Constant(self.Lj[j, :, :]) * h[i,j])
                        tmp += (I*hI[i])
                        D_coll[i] = tmp
                        Dsum += tmp
                    D = cp.hstack([D_coll[i]for i in range(self.P)])
                    term1 = cp.square(cp.norm((Y - D @ X), 'fro'))
                    term2 = cp.square(cp.norm(h, 'fro')*lambda_)
                    term3 = cp.square(cp.norm(hI, 'fro')*lambda_)
                    obj = cp.Minimize(term1 + term2 + term3)

                else:
                    # Init the variables
                    hI = cp.Variable((self.P, self.J))
                    hS = cp.Variable((self.P, self.J))
                    hH = cp.Variable((self.P, 1))
                    hH.value, hS.value, hI.value = h_opt ##################### OCCHIO
                    for i in range(0,self.P):
                        tmp =  cp.Constant(np.zeros((self.M, self.M)))
                        for j in range(0,self.J):
                            tmp += ((cp.Constant(self.Luj[j, :, :])*hS[i,j]) + (cp.Constant(self.Ldj[j, :, :])*hI[i,j]))
                        tmp += (I*hH[i])
                        D_coll[i] = tmp
                        Dsum += tmp
                    D = cp.hstack([D_coll[i] for i in range(self.P)])
        
                    term1 = cp.square(cp.norm((Y - D @ X), 'fro'))
                    term2 = cp.square(cp.norm(hI, 'fro')*lambda_)
                    term3 = cp.square(cp.norm(hS, 'fro')*lambda_)
                    term4 = cp.square(cp.norm(hH, 'fro')*lambda_)
                    obj = cp.Minimize(term1 + term2 + term3 + term4)

                # Define the constraints
                constraints = [D_coll[i] >> 0 for i in range(self.P)] + \
                                [(cp.multiply(self.c, I) - D_coll[i]) >> 0 for i in range(self.P)] + \
                                [(Dsum - cp.multiply((self.c - self.epsilon), I)) >> 0, (cp.multiply((self.c + self.epsilon), I) - Dsum) >> 0]

                prob = cp.Problem(obj, constraints)
                prob.solve(solver=eval(f'cp.{solver}'), verbose=False)

                # Dictionary Update
                D = D.value
                if self.dictionary_type in ["joint", "edge_laplacian"]:
                    h_opt = [h_opt[0] + step_h*(h.value - h_opt[0]),
                             h_opt[1] + step_h*(hI.value - h_opt[1])]
                else:
                    h_opt = [h_opt[0] + step_h*(hH.value-h_opt[0]),
                             h_opt[1] + step_h*(hS.value-h_opt[1]), 
                             h_opt[2] + step_h*(hI.value-h_opt[2])]

                # OMP Step
                X_te_tmp, X_tr_tmp = sparse_transform(D, self.K0, self.Y_test, self.Y_train)
                # Sparse Representation Update
                X_tr = X_tr + step_x*(X_tr_tmp - X_tr)
                X_te = X_te + step_x*(X_te_tmp - X_te)

                # Error Update
                error_train = nmse(D, X_tr, self.Y_train, self.m_train)
                error_test = nmse(D, X_te, self.Y_test, self.m_test)

                hist.append(error_test)
                
                # Error Storing
                if (error_train < self.min_error_train) and (abs(error_train) > np.finfo(float).eps) and (abs(error_train - self.min_error_train) > tol):
                    self.X_opt_train = X_tr
                    self.min_error_train = error_train

                if (error_test < self.min_error_test) and (abs(error_test) > np.finfo(float).eps) and (abs(error_test - self.min_error_test) > tol):
                    self.h_opt = h_opt
                    self.D_opt = D
                    self.X_opt_test = X_te
                    self.min_error_test = error_test
                    pat_iter = 0

                    if verbose == 1:
                        print("New Best Test Error:", self.min_error_test)
                else:
                    pat_iter += 1

                iter_ += 1
        
        else:

            # Fourier Dictionary Benchmark
            _, self.D_opt = sla.eigh(self.L)
            self.X_opt_test, self.X_opt_train = sparse_transform(self.D_opt, self.K0, self.Y_test, self.Y_train)

            # Error Updating
            self.min_error_train = nmse(self.D_opt, self.X_opt_train, self.Y_train, self.m_train)
            self.min_error_test= nmse(self.D_opt, self.X_opt_test, self.Y_test, self.m_test)
            
        return self.min_error_test, self.min_error_train, hist
    

    def _aux_matrix_update(self, X):

        I = [np.eye(self.M)]
        if self.dictionary_type=="separated":
            LLu = [lu for lu in self.Luj]
            LLd = [ld for ld in self.Ldj]
            LL = np.array(I+LLu+LLd)
        else:
            LL = np.array(I + [l for l in self.Lj])

        P_aux = np.array([LL@X[(i*self.M): ((i+1)*self.M), :] for i in range(self.P)])
        self.P_aux = rearrange(P_aux, 'b h w c -> (b h) w c')
    
    def topological_dictionary_learn_qp(self,
                                        lambda_: float = 1e-3, 
                                        max_iter: int = 10, 
                                        patience: int = 10,
                                        tol: float = 1e-7,
                                        solver: str = 'MOSEK',
                                        step_h: float = 1.,
                                        step_x: float = 1.,
                                        verbose: bool = False) -> Tuple[np.ndarray, np.ndarray, List[float]]:
        
        # Define hyperparameters
        iter_, pat_iter = 1, 0
        hist = []

        if self.dictionary_type != "fourier":
        
            # Init the the sparse representation
            h_opt = np.hstack([h.flatten() for h in self.h_opt]).reshape(-1,1)
            X_tr = self.X_opt_train
            X_te = self.X_opt_test
            reg = lambda_ * np.eye(self.P*(2*self.J+1))
            I_s = cp.Constant(np.eye(self.P))
            i_s = cp.Constant(np.ones((self.P,1)))
            B = cp.Constant(self.B)

            while pat_iter < patience and iter_ <= max_iter:

                # Init variables and parameters
                h = cp.Variable((self.P*(2*self.J+1), 1))
                self._aux_matrix_update(X_tr)
                h.value = h_opt

                Q = cp.Constant(np.einsum('imn, lmn -> il', self.P_aux, self.P_aux) + reg)
                l = cp.Constant(np.einsum('mn, imn -> i', self.Y_train, self.P_aux))

                # Quadratic term
                term2 = cp.quad_form(h, Q, assume_PSD = True)
                # Linear term
                term1 = l@h
                term1 = cp.multiply(-2, term1)[0]
                
                obj = cp.Minimize(term2+term1)

                # Define the constraints
                cons1 = cp.kron(I_s, B)@h
                cons2 = cp.kron(i_s.T, B)@h
                constraints = [cons1 >= 0] + \
                                [cons1 <= self.c] + \
                                [cons2 >= (self.c - self.epsilon)] + \
                                [cons2 <= (self.c + self.epsilon)]

                prob = cp.Problem(obj, constraints)
                prob.solve(solver=eval(f'cp.{solver}'), verbose=False)

                # Update the dictionary

                if self.dictionary_type in ["joint", "edge_laplacian"]:
                    h_list = _split_coeffs(h, self.P, self.J)
                    D = generate_dictionary(h.value, self.P, self.Lj)                      
                    h_opt = h_opt + step_h*(h.value - h_opt)
                else:

                    h_list = _split_coeffs(h, self.P, self.J, sep=True)
                    D = generate_dictionary(h_list, self.P, self.Luj, self.Ldj)                                
                    h_opt = h_opt + step_h*(h.value - h_opt)


                # OMP Step
                X_te_tmp, X_tr_tmp = sparse_transform(D, self.K0, self.Y_test, self.Y_train)
                # Sparse Representation Update
                X_tr = X_tr + step_x*(X_tr_tmp - X_tr)
                X_te = X_te + step_x*(X_te_tmp - X_te)

                # Error Update
                error_train = nmse(D, X_tr, self.Y_train, self.m_train)
                error_test = nmse(D, X_te, self.Y_test, self.m_test)

                hist.append(error_test)
                
                # Error Storing
                if (error_train < self.min_error_train) and (abs(error_train) > np.finfo(float).eps) and (abs(error_train - self.min_error_train) > tol):
                    self.X_opt_train = X_tr
                    self.min_error_train = error_train

                if (error_test < self.min_error_test) and (abs(error_test) > np.finfo(float).eps) and (abs(error_test - self.min_error_test) > tol):
                    self.h_opt = h_list if self.dictionary_type == 'separated' else h_opt
                    self.D_opt = D
                    self.X_opt_test = X_te
                    self.min_error_test = error_test
                    pat_iter = 0

                    if verbose == 1:
                        print("New Best Test Error:", self.min_error_test)
                else:
                    pat_iter += 1

                iter_ += 1
        
        else:
            pass

        return self.min_error_test, self.min_error_train, hist
    
    @save_results
    def learn_upper_laplacian(self,
                              Lu_new: np.ndarray = None,
                              filter: np.ndarray = 1,
                              h_prior: np.ndarray = None,
                              lambda_: float = 1e-3, 
                              max_iter: int = 10, 
                              patience: int = 10,
                              tol: float = 1e-7,
                              step_h: float = 1.,
                              step_x: float = 1.,
                              mode: str = "optimistic",
                              verbose: bool = False,
                              warmup: int = 0,
                              QP=False,
                              cont=False):
    
        assert step_h<1 or step_h>0, "You must provide a step-size between 0 and 1."
        assert step_x<1 or step_x>0, "You must provide a step-size between 0 and 1."
        assert (mode=="optimistic") or (mode=="pessimistic"), f'{mode} is not a legal mode: \"optimistic\" or \"pessimistic\" are the only ones allowed.'
        
        # Check if we are executing the first recursive iteration
        if np.all(Lu_new == None):
            T = self.B2.shape[1]
            if mode=="optimistic":
                filter = np.ones(T)
                self.warmup=0  
            else:
                filter = np.zeros(T)
                self.update_Lu(np.zeros(self.Lu.shape)) # start with an "empty" upper Laplacian
                self.warmup = warmup
        else:
            self.update_Lu(Lu_new)

        self.init_dict(h_prior=h_prior,
                       mode="only_X")

        if QP:
            _, _, hist = self.topological_dictionary_learn_qp(lambda_=lambda_,
                                                            max_iter=max_iter,
                                                            patience=patience,
                                                            tol=tol,
                                                            step_h=step_h,
                                                            step_x=step_x,
                                                            solver='GUROBI')
        else:
            _, _, hist = self.topological_dictionary_learn(lambda_=lambda_,
                                                            max_iter=max_iter,
                                                            patience=patience,
                                                            tol=tol,
                                                            step_h=step_h,
                                                            step_x=step_x)
                        
        self.history.append(hist)
        search_space = np.where(filter == 1) if mode=="optimistic" else np.where(filter == 0)   
        sigmas = pd.DataFrame({"idx": search_space[0]})

        sigmas["sigma"] = sigmas.idx.apply(lambda _: filter)
        if mode=="optimistic":
            sigmas["sigma"] = sigmas.apply(lambda x: _indicator_matrix(x), axis=1)
        else:
            sigmas["sigma"] = sigmas.apply(lambda x: _indicator_matrix_rev(x), axis=1)
        sigmas["Luj"] = sigmas.apply(lambda x: _compute_Luj(x, self.B2, self.J), axis=1)
        sigmas["D"] = sigmas.apply(lambda x: generate_dictionary(self.h_opt, self.P, x.Luj, self.Ldj), axis=1)
        sigmas["X"] = sigmas.D.apply(lambda x: sparse_transform(x, self.K0, self.Y_test))
        sigmas["NMSE"] = sigmas.apply(lambda x: nmse(x.D, x.X, self.Y_test, self.m_test), axis=1)
        
        if self.warmup>0:
            candidate_error = sigmas.NMSE.min() - np.finfo(float).eps
            self.warmup-=1
        else:
            candidate_error = sigmas.NMSE.min()
        idx_min = sigmas.NMSE.idxmin()

        
        if candidate_error < self.min_error_test:
            S = sigmas.sigma[idx_min]
            Lu_new = self.B2 @ S @ self.B2.T
            filter = np.diagonal(S)

            if verbose:
                if mode=="optimistic":
                    print(f'Removing 1 triangle from topology... \n ... New min test error: {candidate_error} !')
                else:
                    print(f'Adding 1 triangle to topology... \n ... New min test error: {candidate_error} !')

            return self.learn_upper_laplacian(h_prior=self.h_opt,
                                              Lu_new=Lu_new,
                                              filter=filter,
                                              lambda_=lambda_,
                                              max_iter=max_iter,
                                              patience=patience,
                                              tol=tol,
                                              step_h=step_h,
                                              step_x=step_x,
                                              mode=mode,
                                              verbose=verbose,
                                              QP=QP,
                                              cont=cont)
        
        # For the last recursions of "pessimistic" mode try some recursion of the "optimistic"
        # to remove the warm-up randomly-added triangles
        if mode == "pessimistic" and not cont:
            return  self.learn_upper_laplacian(h_prior=self.h_opt,
                                              Lu_new=Lu_new,
                                              filter=filter,
                                              lambda_=lambda_,
                                              max_iter=max_iter,
                                              patience=patience,
                                              tol=tol,
                                              step_h=step_h,
                                              step_x=step_x,
                                              mode="optimistic",
                                              verbose=verbose,
                                              QP=QP,
                                              cont=True)
        
        # Then after we added triangles and removed the randomly added ones, continue adding!
        elif mode != "pessimistic" and cont:

            print("Ce provo!")
            return  self.learn_upper_laplacian(h_prior=self.h_opt,
                                              Lu_new=Lu_new,
                                              filter=filter,
                                              lambda_=lambda_,
                                              max_iter=max_iter,
                                              patience=patience,
                                              tol=tol,
                                              step_h=step_h,
                                              step_x=step_x,
                                              mode="pessimistic",
                                              verbose=verbose,
                                              QP=QP,
                                              cont=cont,
                                              warmup=warmup)  
                  
        self.B2 = self.B2@np.diag(filter)
        return self.min_error_test, self.history, self.Lu, self.B2


